## PLS
eerste grote stappen in de pls

In [18]:
# loading dataframes 

File_url_d = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/managed_portfolios_anom_d_50.csv'

r_daily = pd.read_csv(File_url_d)

# set date to datetime format and set the date to the index 

r_daily['date'] = pd.to_datetime(r_daily['date'])
r_daily.set_index('date', inplace=True)

# missing list = ['dur', 'divg', 'invaci', 'ipo'] -> deze worden gemaakt uit daily stock data doormiddel van een signals list?

# following the procedure in the paper, if there are observations missing we set them to 0. 

r_daily.fillna(0, inplace=True)





In [66]:
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
import calendar

# Define initial parameters
initial_start_date = pd.to_datetime('1967-07-01')
initial_training_years = 10

# Define factors
factors = [
    'rme', 're_ew', 'r_size', 'r_value', 'r_prof', 'r_valprof', 'r_fscore',
    'r_debtiss', 'r_repurch', 'r_nissa', 'r_accruals', 'r_growth', 'r_aturnover',
    'r_gmargins', 'r_divp', 'r_ep', 'r_cfp', 'r_noa', 'r_inv', 'r_invcap',
    'r_igrowth', 'r_sgrowth', 'r_lev', 'r_roaa', 'r_roea', 'r_sp', 'r_gltnoa',
    'r_indmom', 'r_valmom', 'r_valmomprof', 'r_shortint', 'r_mom12', 'r_momrev',
    'r_lrrev', 'r_valuem', 'r_nissm', 'r_sue', 'r_roe', 'r_rome', 'r_roa',
    'r_strev', 'r_ivol', 'r_betaarb', 'r_season', 'r_indrrev', 'r_indrrevlv',
    'r_indmomrev', 'r_ciss', 'r_price', 'r_age', 'r_shvol'
]

# Create an empty dataframe for monthly PLS returns
pls_return_df = pd.DataFrame(columns=['date'] + [f"PLS{i+1}" for i in range(len(factors))])

# Initialize an empty list to store PLS weights
pls_weights_list = []

# Loop through years and months
for year in range(1973, 1975):
    for month in range(6, 13) if year == 1973 else range(1, 13):

        # Define start and end dates
        last_day = calendar.monthrange(year, month)[1]
        end_date = pd.to_datetime(f'{year}-{month}-{last_day}')

        # For the expanding window, start date becomes initial start date
        start_date = initial_start_date

        # Select data range for PLS analysis
        pls_data = r_daily.loc[start_date:end_date]

        # Separate features (X) and target (y)
        X = pls_data[factors]
        y = pls_data['r_mom12']

        # Fit PLS model
        pls = PLSRegression(n_components=10)  # Adjust number of components as needed
        pls.fit(X, y)

        # Calculate predicted values
        predictions = pls.predict(X)

        # Extract and store PLS weights
        pls_weights = pd.DataFrame(pls.x_weights_, index=factors, columns=[f"PLS{i+1}" for i in range(pls.x_weights_.shape[1])])
        pls_weights_list.append(pls_weights)

        # Append the monthly PLS returns to the dataframe
        pls_return_data = {'date': end_date}
        for f in range(len(factors)):
            pls_return_data[f"PLS{f+1}"] = predictions[f]
        pls_return_df = pls_return_df.append(pls_return_data, ignore_index=True)

# Print the resulting dataframe
print("PLS Returns:")
print(pls_return_df)

# Print PLS weights
for i, weights in enumerate(pls_weights_list, start=1):
    print(f"PLS Weights for month {i}:")
    print(weights)


/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_7617/4149981333.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pls_return_df = pls_return_df.append(pls_return_data, ignore_index=True)
/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_7617/4149981333.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pls_return_df = pls_return_df.append(pls_return_data, ignore_index=True)
/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_7617/4149981333.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pls_return_df = pls_return_df.append(pls_return_data, ignore_index=True)
/var/folders/hx/jlx4phmj419bwmhwf_bz4qqm0000gn/T/ipykernel_7617/4149981333.py:60: FutureWarning: The frame.append method is deprec

PLS Returns:
         date      PLS1      PLS2      PLS3      PLS4      PLS5      PLS6  \
0  1973-06-30  0.002139  0.004626  0.000814  0.003847  0.005026  0.000667   
1  1973-07-31  0.002154  0.004574  0.000795  0.003809  0.005026  0.000691   
2  1973-08-31  0.002255  0.004811  0.000881  0.003696  0.004940  0.000600   
3  1973-09-30  0.002243  0.004687  0.000830  0.003710  0.004907  0.000652   
4  1973-10-31  0.002194  0.004708  0.000877  0.003743  0.004978  0.000687   
5  1973-11-30  0.002143  0.004708  0.000902  0.003774  0.004997  0.000648   
6  1973-12-31  0.002067  0.004734  0.000923  0.003884  0.005024  0.000610   
7  1974-01-31  0.002164  0.004709  0.000886  0.003791  0.005003  0.000660   
8  1974-02-28  0.002166  0.004727  0.000900  0.003813  0.004996  0.000621   
9  1974-03-31  0.002154  0.004756  0.000902  0.003851  0.005008  0.000577   
10 1974-04-30  0.002166  0.004756  0.000885  0.003821  0.004964  0.000576   
11 1974-05-31  0.002199  0.004734  0.000848  0.003768  0.004925